## Import Initial Data

In [1]:
import pandas as pd

In [2]:
school_load = "Resources/schools_complete.csv"
student_load = "Resources/students_complete.csv"

In [3]:
school_data = pd.read_csv(school_load)
student_data = pd.read_csv(student_load)

In [4]:
data = pd.merge(student_data, school_data, how = "left", on = ["school_name", "school_name"])

## District Summary

In [5]:
schools_total = data["school_name"].nunique()

In [6]:
students_total = data["Student ID"].count()

In [7]:
budget_list = data["budget"].unique()
budget_total = (budget_list).sum()

In [8]:
math_avg = data["math_score"].mean()

In [9]:
read_avg = data["reading_score"].mean()

In [10]:
math_pass_raw = data.loc[data["math_score"] >= 70, ["math_score"]].count()
math_pass_mean = math_pass_raw.mean()
math_pass = math_pass_mean / students_total * 100

In [11]:
read_pass_raw = data.loc[data["reading_score"] >= 70, ["reading_score"]].count()
read_pass_mean = read_pass_raw.mean()
read_pass = read_pass_mean / students_total * 100

In [12]:
ovr_pass_raw = data.loc[(data["reading_score"] >= 70) & (data["math_score"] >= 70), ["reading_score"]].count()
ovr_pass_mean = ovr_pass_raw.mean()
ovr_pass = ovr_pass_mean / students_total * 100

In [13]:
district_df = pd.DataFrame([[schools_total, students_total, budget_total, math_avg, read_avg, math_pass, read_pass, ovr_pass]], 
                           columns=["Total Schools", "Total Students", "Total Budget", "Average Math Score", 
                                    "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [14]:
school_type = data.iloc[:, [4,8]]
df_1 = school_type.drop_duplicates()

In [15]:
df_1["Total Students"] = data.groupby(["school_name"])["student_name"].transform(len)

C:\Users\zachr\AppData\Local\Temp\ipykernel_20268\1324822495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["Total Students"] = data.groupby(["school_name"])["student_name"].transform(len)


In [16]:
school_budget = data.iloc[:, [4, 10]]
df_2 = school_budget.drop_duplicates

In [17]:
df_3 = pd.merge(df_1, df_2(), how = "inner", on = "school_name")

In [18]:
df_3["Per Student Budget"] = (df_3["budget"]) / (df_3["Total Students"])

In [19]:
math_avg_school = data.groupby(["school_name"])["math_score"].mean

In [20]:
df_4 = pd.merge(df_3, math_avg_school(), how = "inner", on = "school_name")

In [21]:
read_avg_school = data.groupby(["school_name"])["reading_score"].mean

In [22]:
df_5 = pd.merge(df_4, read_avg_school(), how = "inner", on = "school_name")

In [23]:
math_pass_school = data.iloc[:, [4, 6]]
math_pass_school

math_pass_school_percent = math_pass_school[math_pass_school.math_score >= 70].groupby("school_name").count

In [24]:
df_6 = pd.merge(df_5, math_pass_school_percent(), how = "inner", on = "school_name")
df_6["% Passing Math"] = (df_6["math_score_y"]) / (df_6["Total Students"]) * 100


In [25]:
read_pass_school = data.iloc[:, [4, 5]]
read_pass_school

read_pass_school_percent = read_pass_school[read_pass_school.reading_score >= 70].groupby("school_name").count

In [26]:
df_7 = pd.merge(df_6, read_pass_school_percent(), how = "inner", on = "school_name")
df_7["% Passing Reading"] = (df_7["reading_score_y"]) / (df_7["Total Students"]) * 100

In [27]:
df_8 = data.loc[(data["math_score"] >= 70) & (data["reading_score"] >= 70), ["school_name"]].value_counts()

In [28]:
df_9 = pd.merge(df_7, df_8.to_frame(), how = "inner", on = "school_name")
df_9["% Overall Passing"] = (df_9[0]) / (df_9["Total Students"]) * 100

del df_9["math_score_y"]
del df_9["reading_score_y"]
del df_9[0]

In [29]:
df_10 = df_9.rename(columns={"type":"School Type", 
                            "budget":"Total School Budget",
                            "math_score_x":"Average Math Score",
                            "reading_score_x":"Average Reading Score"})

In [30]:
df_10 = df_10.set_index("school_name")
df_10.index.name = None
df_10 = df_10.sort_index(ascending = True)
df_10

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools

In [31]:
df_10.sort_values(by = ["% Overall Passing"], ascending = False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools

In [32]:
df_10.sort_values(by = ["% Overall Passing"], ascending = True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [33]:
# This part tripped me up a lot, I gave it my best effort trying a bunch of different methods
m_grades = data.groupby(["grade", "school_name"])["math_score"].mean().to_frame("Math Grade")
m_grades

Math Grade
grade school_name                      
10th  Bailey High School      76.996772
      Cabrera High School     83.154506
      Figueroa High School    76.539974
      Ford High School        77.672316
      Griffin High School     84.229064
      Hernandez High School   77.337408
      Holden High School      83.429825
      Huang High School       75.908735
      Johnson High School     76.691117
      Pena High School        83.372000
      Rodriguez High School   76.612500
      Shelton High School     82.917411
      Thomas High School      83.087886
      Wilson High School      83.724422
      Wright High School      84.010288
11th  Bailey High School      77.515588
      Cabrera High School     82.765560
      Figueroa High School    76.884344
      Ford High School        76.918058
      Griffin High School     83.842105
      Hernandez High School   77.136029
      Holden High School      85.000000
      Huang High School       76.446602
      Johnson High School     77.491653
      Pena High School        84.328125
      Rodriguez High School   76.395626
      Shelton High School     83.383495
      Thomas High School      83.498795
      Wilson High School      83.195326
      Wright High School      83.836782
12th  Bailey High School      76.492218
      Cabrera High School     83.277487
      Figueroa High School    77.151369
      Ford High School        76.179963
      Griffin High School     83.356164
      Hernandez High School   77.186567
      Holden High School      82.855422
      Huang High School       77.225641
      Johnson High School     76.863248
      Pena High School        84.121547
      Rodriguez High School   77.690748
      Shelton High School     83.778976
      Thomas High School      83.497041
      Wilson High School      83.035794
      Wright High School      83.644986
9th   Bailey High School      77.083676
      Cabrera High School     83.094697
      Figueroa High School    76.403037
      Ford High School        77.361345
      Griffin High School     82.044010
      Hernandez High School   77.438495
      Holden High School      83.787402
      Huang High School       77.027251
      Johnson High School     77.187857
      Pena High School        83.625455
      Rodriguez High School   76.859966
      Shelton High School     83.420755
      Thomas High School      83.590022
      Wilson High School      83.085578
      Wright High School      83.264706

## Reading Score by Grade

In [34]:
r_grades = data.groupby(["grade", "school_name"])["reading_score"].mean().to_frame("Reading Grade")
r_grades

Reading Grade
grade school_name                         
10th  Bailey High School         80.907183
      Cabrera High School        84.253219
      Figueroa High School       81.408912
      Ford High School           81.262712
      Griffin High School        83.706897
      Hernandez High School      80.660147
      Holden High School         83.324561
      Huang High School          81.512386
      Johnson High School        80.773431
      Pena High School           83.612000
      Rodriguez High School      80.629808
      Shelton High School        83.441964
      Thomas High School         84.254157
      Wilson High School         84.021452
      Wright High School         83.812757
11th  Bailey High School         80.945643
      Cabrera High School        83.788382
      Figueroa High School       80.640339
      Ford High School           80.403642
      Griffin High School        84.288089
      Hernandez High School      81.396140
      Holden High School         83.815534
      Huang High School          81.417476
      Johnson High School        80.616027
      Pena High School           84.335938
      Rodriguez High School      80.864811
      Shelton High School        84.373786
      Thomas High School         83.585542
      Wilson High School         83.764608
      Wright High School         84.156322
12th  Bailey High School         80.912451
      Cabrera High School        84.287958
      Figueroa High School       81.384863
      Ford High School           80.662338
      Griffin High School        84.013699
      Hernandez High School      80.857143
      Holden High School         84.698795
      Huang High School          80.305983
      Johnson High School        81.227564
      Pena High School           84.591160
      Rodriguez High School      80.376426
      Shelton High School        82.781671
      Thomas High School         83.831361
      Wilson High School         84.317673
      Wright High School         84.073171
9th   Bailey High School         81.303155
      Cabrera High School        83.676136
      Figueroa High School       81.198598
      Ford High School           80.632653
      Griffin High School        83.369193
      Hernandez High School      80.866860
      Holden High School         83.677165
      Huang High School          81.290284
      Johnson High School        81.260714
      Pena High School           83.807273
      Rodriguez High School      80.993127
      Shelton High School        84.122642
      Thomas High School         83.728850
      Wilson High School         83.939778
      Wright High School         83.833333

## Scores by School Spending

In [35]:
bins = [0, 585, 630, 645, 680]
bin_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [36]:
data_copy = df_10

In [37]:
data_copy["Spending Ranges Per Student"] = pd.cut(data_copy["Per Student Budget"], bins, labels = bin_labels)

In [38]:
budget_breakdown = data_copy.groupby(["Spending Ranges Per Student"]).mean()

In [39]:
budget_breakdown_final = budget_breakdown.filter(["Average Math Score", "Average Reading Score", "% Passing Math",
                                                 "% Passing Reading", "% Overall Passing"], axis = 1)
budget_breakdown_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges Per Student,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [40]:
bins_2 = [0, 1000, 2000, 5000]
bin_labels_2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [41]:
data_copy_2 = df_10

In [42]:
data_copy_2["School Size"] = pd.cut(data_copy_2["Total Students"], bins_2, labels = bin_labels_2)

In [43]:
size_breakdown = data_copy_2.groupby(["School Size"]).mean()

In [44]:
size_breakdown_final = size_breakdown.filter(["Average Math Score", "Average Reading Score", "% Passing Math",
                                                 "% Passing Reading", "% Overall Passing"], axis = 1)
size_breakdown_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [45]:
data_copy_3 = df_10

In [46]:
type_breakdown = data_copy_3.groupby(["School Type"]).mean()

In [47]:
type_breakdown_final = type_breakdown.filter(["Average Math Score", "Average Reading Score", "% Passing Math",
                                                 "% Passing Reading", "% Overall Passing"], axis = 1)
type_breakdown_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
